# Projeto 1: Unsupervised Learning

## Grupo
Nomes: Camila Moraes Brito, Gabriel de Freitas Garcia  
Ra: , 216179

# Tratamento dos Dados

Foram usados dois datasets em nosso projeto, eles podem ser encontrados em: "../data". O primerio foi um dataset de duas dimensões fornecido pela professora, o segundo foi um dataset sobre ataques cardíacos encontrado em [Dataset ataque cardíaco](https://www.kaggle.com/fedesoriano/stroke-prediction-dataset). Em ambos os datasets optou-se por fazer fazer a leitura dos dados e depois seriaizá-los em formato JSON para evitar execuções desnecessárias do algoritmo de recuperação dos dados e por JSON ser mais fácil de recuperar que os formatos fornecidos. O padrão dos objetos JSON será exibido a seguir:  
![teacher data](teacherData.png)
![stroke data](strokeData.png)


Os notebooks onde este tratamento é feito se encontram em [Teacher Data](../Notebooks/parsingData1.ipynb) e [Stroke Data](../Notebooks/parsingData1.ipynb). Para o dataset fornecido o tratamento foi apenas verificar se havia dados não numéricos entre os dados corretos, por meio de tratamento de exceção. Para o outro dataset o tratamento será explicado em detalhes mais abaixo. Nesta etapa nenhum dado foi normalizado.

O dataset sobre ataques cardíacos possui 12 features, uma delas, porém, o ID, representa um identificador único de cada indivíduo. Essa feature abrange uma gama muito grande de valores que não possui, provavelmente, nenhum significado e não serve para classificação dos elementos, ou seja, não é relevante para a análise e pode prejudicá-la, portanto optou-se por desprezá-la. As outras features e o tipo de dado que contem são: gênero - classificação (Homem, mulher ou outro), idade - inteiro, hipertensão - booleano, doença cardíaca - booleano, já foi casado - booleano, tipo de trabalho - classificação (Criança, desempregado, autônomo, funcionário privado ou funcionário público), tipo de habitação - classificação (Urbana ou Privada), nível médio de glicose - número contínuo, índice de massa corpórea - número contínuo, status de fumo - classificação (nunca fumou, ex-fumante, fumante ou desconhecido), ataque cardíaco - booleano.

Tendo em vista que os métodos de aprendizado não supervisionado utilizados precisam de números para trabalhar, todos os dados que não fossem numéricos precisaram ser adaptados, utilizando uma codificação desenvolvida por um dos membros da equipe que será explicada abaixo

Gênero:  
Mulher : 1  
Outro : 2  
Homem : 3  

Essa decisão foi tomada com base numa ideia de proximidade, Mulher e homem são diferentes e opostos em termos classificatórios já outro, o desenvolvedor da codificação optou por não entrar em discussões que fogem ao escopo do projeto, porém interpretou que está igualmente perto de homem ou mulher, sem privilegiar nenhum. Por isso foi escolhidos os valores referenciados acima

Tipo de trabalho:  
Criança : 0  
Desempregado : 1  
Autônomo : 2  
Funcionário privado : 3  
Funcionário público : 4

Neste caso, a classificação se deu pelo nível de formalidade da ocupação, quanto mais formal, maior o número, crianças ficam com zero pois não podem trabalhar. Desta forma, tipos de emprego mais próximos, em questão de formalidade, ficam mais próximos e os muito díspares ficam mais distantes entre si.

Tipo de habitação:  
Rural : 0  
Urbano : 1  

Essa feature é binária, portanto, 0 e 1 foi a escolha natural para esse caso, qualquer outros dois valores diferentes poderiam ser usados, esses porém mantém a consistência com as dimensões booleanas do dataset.

Status de fumo:
Nunca fumou : 0  
Parou de fumar: 1  
Fuma : 2  
Desconhecido : -1  

Neste caso, fumantes e não fumantes são opções diametralmente opostas, com ex-fumante estando entre as duas opções por já ter fumado, porém não fumar mais. Há uma quarta opção, desconhecido, que significa que esse dado não está disponível e deixou-se em -1, uma vez que todos os dados são positivos. Essa escolha é consistente com outras dimensões, que optou-se por marcar -1 toda vez que houvesse uma célula vazia.

Para as outras dimensões, manteve-se o número que estava na entrada, apenas tomando o cuidado de verificar se era um valor válido, se não, registrou-se -1. A dimensão "Já foi casado" ao invés de 0 e 1, como nas outras colunas booleanas, tinha "sim e não", que foram devidamente substituídas por 0 e 1, mantendo-se a consistência dos dados.

As leituras nesse dataset foram feitas por meio da biblioteca Pandas, que possui uma série de funções adequadas para leitura e manipulação de dados, o que inclui leitura de arquivos CSV (comma separated values), o formato de entrada deste dataset. 

O dataset fornecido foi lido por meio das funções padrão de leitura de arquivos da liguagem Python, devido ao seu formato de dois números por linha separados por espaço, sem cabeçalho.

Ambas as fontes de dados foram separadas em conjuntos de aprendizado e teste de tamanho aproximadamente 90% e 10% respectivamente. Isso se deu por meio de escolha aleatória de 10% dos pontos de cada dataset, algoritmo que mantém a distribuição da amostra retirada. Um dos códigos feitos para isso será mostrado abaixo, o código para o outro dataset é análogo e pode ser encontrado junto com todo o código desta etapa do projeto nos notebooks anteriormente anexados.  
Um campo cluster, com valor -1, foi adicionado a cada um dos pontos em ambos datasets para utilização em etapas posteriores.

In [ ]:
try:
    f = open("../data/cluster.dat")
except:
    print("Houston we've got a problem")
dataset = []
testset = []
for i in f:
    rVar = random.randint(1,10)
    stringList = i.split()
    try:
        dicti = {
            "x" : float(stringList[0]),
            "y" : float(stringList[1]),
            "cluster" : -1
        }
    except ValueError:
        print("You had a Value Error")
        break
    except:
        print("You got another Error")
        break
    if(rVar == 5):
        testset.append(dicti)
    else:
        dataset.append(dicti)
f.close()

# Kmeans

O código desta etapa se encontra no notebook [Kmeans](../Notebooks/kMeans.ipynb).

A primeira coisa feita nesta etapa foi a leitura dos dados no JSON produzido na etapa anterior. O dicionário lido está num formato facilmente compreensível para humanos, porém difícil de se trabalhar na implementação, portanto para esta etapa ele foi salvo numa nova estrutura de dados, um dicionário com dois campos: cluster, onde fica salvo a lista com o código do cluster de cada ponto respectivamente e o segundo campo é matrix, uma matriz em que cada linha representa uma feature e cada coluna representa o valor da feature para o ponto i. O código será mostrado a seguir. Esta estrutura de dados se provou problemática por não manter todas as dimensões de cada ponto numa estrutura só, mas em listas separadas, o que contraria a interface de muitas das funções das bibliotecas usadas. Ela também não usa np.arrays. e sim listas python, que são muito mais ineficientes e difíceis de trabalhar com as funções da biblioteca Numpy, exigindo que várias conversões fossem feitas ao longo do código, o que é altamente ineficiente. Ela foi mantida apenas para não ser necessário refatorar todo o código que já tinha sido feito usando ela, porém foi substituída na etapa seguinte.

In [ ]:
def getDataset(dataset):
    cleanDataset = {
        "matrix" : [],
        "cluster": [],
    }
    for i in dataset[0]:
        if i != "cluster":
            cleanDataset["matrix"].append(getList(dataset,i))
        elif i == "cluster":
            cleanDataset["cluster"] = getList(dataset,"cluster")
    return cleanDataset
exist = True
try:
    f = open("../data/teacherData.json","r")
    stringSet = f.read()
    f.close()
except:
    print("Arquivo não existe")
    exist = False
if(exist):
    dataset = json.loads(stringSet)
    nDataset = getDataset(dataset)
    plt.scatter(nDataset["matrix"][0],nDataset["matrix"][1])
    plt.title("Original Data")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.savefig("../images/originalData1.png")

O código da nossa implementação do Kmeans será anexado em seguida. A distância utilizada foi a distância Euclidiana e ass estratégias de inicialização foram Forgy e - random partition -, que serão explicadas mais adiante. Todas as funções auxiliares podem ser encontradas no notebook anteriormente mencionado e não serão anexadas para manter a concisão deste relatório.

In [1]:
def kmeans(dataset,k):
    centers = [] 
    centers = initialize(dataset["matrix"],k)
    for i in range(len(dataset["cluster"])):
        dataset["cluster"][i] = getCluster(getPoint(dataset["matrix"],i),centers)
    k = 0
    oldCenters = copy.deepcopy(centers)
    while(True):
        for i in range(len(centers)):
            newCenter = getCenter(getClusterPoints(dataset,i))
            if newCenter == None:
                centers[i] = [0]*len(centers[i])
                continue
            centers[i] = newCenter
        for i in range(len(dataset["cluster"])):
            dataset["cluster"][i] = getCluster(getPoint(dataset["matrix"],i),centers)
        k+=1
        if k > MAXIT:
            break
        elif (not change(centers,oldCenters)) and k > 0:
            break
        oldCenters = copy.deepcopy(centers)
    print(k)
    dataset["centers"] = centers
    return dataset

Este código segue os princípios do Kmeans, ele inicializa o dataset seguindo as estratégias adotadas, então a cada iteração ele encontra novos centros para cada cluster e depois atualiza o cluster de cada ponto, parando quando todos os centros de cada cluster não se moverem mais, dentro de uma margem de erro pré estabelecida. Também foi estabelecido um valor máximo de iterações permitidas, para evitar a situação de um loop infinito no caso de um bug passar despercebido no código

## Efeito da normalização

Observou-se que esse algoritmo é altamente sensível à diferença na ordem de grandeza entre as dimensões. O algoritmo produziu resultados diferentes do esparado com os dados não normalizados e serão comparados com e sem normalização para o dataset fornecido, para o outro dataset a análise fica muito mais complicada por não se poder plotar gráficos de pontos do resultado e então optou-se por trabalhar apenas com os dados normalizados

Dados originais  
![Original data](../images/originalData1.jpg) 

Resultado sem normalização  
![No Norm result](../images/clusteredData1n.jpg) 

Elbow sem normalização  
![No Norm Elbow](../images/Elbow1n.jpg) 

Resultado normalizado  
![Norm result](../images/clusteredData1.jpg) 

Elbow normalizado  
![Norm Elbow](../images/Elbow1.jpg) 

Como observado, uma das dimensões é várias ordens de grandeza superior à outra, o que acaba enviesando a distância euclidiana. A imagem possui três grupos claros e distintos, o algoritmo conseguiu encontrar esses três clusters para os dados normalizados, porém encontrou grupos errados para os dados não normalizados. Ambas as análisess de Elbow ficaram com o mesmo formato, porém a função de custo para os dados não normalizados possui uma seção quase vertical que vai até 1e8 entre k = 1 e k = 2, o que não acontece com os dados normalizados, que também apresentam custos menores em toda a curva quando comparados com os dados não normalizados.

Mesmo utilizando a inicialização forgy, que é aleatória, obteve-se o mesmo resultado em todas as execuções e também observou-se uma diferença nas quantidades de iterações que podem ser observadas na tabela a seguir:

### Tabela número de iterações:


k | Não normalizado | Normalizado
- | --------------- | -----------
1 | 2               | 2
2 | 9               | 4
3 | 8               | 4
4 | 12              | 6
5 | 14              | 10
6 | 8               | 7
7 | 10              | 9
8 | 21              | 12
9 | 10              | 10
10| 34              | 10

Para cada k o número de iterações é igual ou menor no normalizado em relação ao não normalizado, chegando a apresentar mais de dez iterações de diferença para alguns ks

Desta forma é possível perceber que a análise com dados normalizados é uma opção melhor do que com dados não normalizados. A estratégia de normalização está no código a seguir:

In [4]:
def normalize(array):
    norm = np.linalg.norm(np.array(array))
    for i in range(len(array)):
        array[i] = array[i]/norm
    return array

Este algoritmo é aplicado para cada lista de features do dataset e a normalização é feita dividindo-se cada elemento da lista pela norma dois da lista.

## Resultados

As imagens com os resultados da aplicação sobre os dados da professora foram apresentados na seção anterior e serão discutidos agora.